In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups_vectorized

## 案例2

In [2]:
# 导入必要的库
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OrdinalEncoder

# 数据集使用了模拟的医疗数据，包含患者的症状特征和是否患病的标签
# 数据格式：[体温(低/中/高), 血压(低/正常/高), 血糖(低/正常/高)]
data = [
["低", "正常", "低", "无病"],
["中", "正常", "正常", "无病"],
["高", "高", "高", "有病"],
["高", "正常", "高", "有病"],
["低", "低", "正常", "无病"],
["中", "高", "高", "有病"],
["低", "正常", "低", "无病"],
["高", "高", "正常", "有病"],
["中", "正常", "正常", "无病"],
["高", "低", "高", "有病"]
]

# 将数据转换为特征和标签
X = np.array([row[:-1] for row in data])  # 特征 (症状)
y = np.array([row[-1] for row in data])   # 标签 (是否患病)

# 使用 OrdinalEncoder 将字符串特征编码为数值，便于模型处理
encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(X)

# 通过 train_test_split 将数据划分为训练集和测试集，确保模型评估的可靠性
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

# 使用 CategoricalNB 实现朴素贝叶斯分类器，适用于离散特征
model = CategoricalNB()  
model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = model.predict(X_test)

# 输出评估结果
print("预测结果:", y_pred)
print("真实标签:", y_test)
print("准确率:", accuracy_score(y_test, y_pred))
print("分类报告:\n", classification_report(y_test, y_pred))

# 示例：预测新患者是否患病，展示其在实际问题中的应用
new_patient = [["高", "正常", "高"]]  # 新患者的症状
new_patient_encoded = encoder.transform(new_patient)  # 编码新患者数据
prediction = model.predict(new_patient_encoded)
print("新患者预测结果:", prediction[0])

预测结果: ['无病' '无病' '有病']
真实标签: ['无病' '无病' '有病']
准确率: 1.0
分类报告:
               precision    recall  f1-score   support

          无病       1.00      1.00      1.00         2
          有病       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

新患者预测结果: 有病


## AODE

In [ ]:
# 导入必要的库
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score

# 定义 AODE 类
class AODE:
	def __init__(self):
		"""
		初始化 AODE 模型。
		self.models 用于存储多个“一对多依赖”模型。
		"""
		self.models = []

	def fit(self, X, y):
		"""
		构建多个“一对多依赖”模型。
		对于每个特征 i，排除该特征后训练一个朴素贝叶斯模型。
		"""
		n_features = X.shape[1]  # 获取特征数量
		for i in range(n_features):
			model = CategoricalNB()  # 使用 Scikit-learn 的朴素贝叶斯分类器
			model.fit(np.delete(X, i, axis=1), y)  # 排除第 i 个特征
			self.models.append(model)
	
	def predict(self, X):
		"""
		对多个模型的结果进行加权平均。
		每个模型基于排除某个特征后的数据进行预测，最终取所有模型结果的平均值。
		"""
		predictions = np.zeros((X.shape[0], len(self.models)))  # 初始化预测结果矩阵
		for i, model in enumerate(self.models):
			predictions[:, i] = model.predict(np.delete(X, i, axis=1))  # 排除第 i 个特征后预测
		return np.round(predictions.mean(axis=1)).astype(int)  # 取平均值并四舍五入为整数

# 模拟学生成绩数据集
"""
数据格式：[学习时间(小时), 作业完成率(%), 课堂参与度(0-5评分), 成绩是否优秀(0/1)]
目标变量是成绩是否优秀（0 表示非优秀，1 表示优秀）。
"""
data = [
[8, 90, 4, 1],  # 优秀
[6, 70, 3, 0],  # 非优秀
[10, 95, 5, 1], # 优秀
[5, 60, 2, 0],  # 非优秀
[7, 80, 4, 1],  # 优秀
[4, 50, 1, 0],  # 非优秀
[9, 85, 5, 1],  # 优秀
[3, 40, 2, 0],  # 非优秀
]

# 将数据转换为特征和标签
X = np.array([row[:-1] for row in data])  # 特征
y = np.array([row[-1] for row in data])   # 标签

# 数据集划分
"""
将数据集划分为训练集和测试集，比例为 7:3。
random_state 固定随机种子以确保结果可复现。
"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 使用 AODE 进行训练和预测
"""
初始化 AODE 模型并进行训练。
在测试集上进行预测，并计算准确率。
"""
aode_model = AODE()
aode_model.fit(X_train, y_train)  # 训练 AODE 模型
aode_pred = aode_model.predict(X_test)  # 在测试集上进行预测

# 输出准确率和预测结果
print("半朴素贝叶斯（AODE）准确率:", accuracy_score(y_test, aode_pred))
print("半朴素贝叶斯（AODE）预测结果:", aode_pred)

半朴素贝叶斯（AODE）准确率: 0.6666666666666666
半朴素贝叶斯（AODE）预测结果: [0 0 0]
